In [1]:
#import packages

import rasterio.features
import geopandas as gpd
import pandas as pd
from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import rasterstats as rs
# import pyproj
# from pyproj import Transformer
# import fiona
# import random
# import scikitlearn
# import sklearn
# import rasterio
import numpy as np
# import geowombat as gw
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [3]:
# #define satellite image, aoi, and training data paths
# raster = "/Users/philipp/BPLA Dropbox/03 Planning/1232-T2-GIS and Remote Sensing-TMO/02_GIS_Data/LandCoverClassificaiton/lcc-trial/21may-lcc.tif"
# raster="/Users/philipp/BPLA Dropbox/08 GIS-data/GIS-data-Riyadh/2022-08-DQ-PNEO-satellite-image/geoserve_orthos-saudi-arabia_2023-02-22_1207/oDIM_PNEO4_202208080738351_PMS-FS_SEN_PWOI_000072103_2_1_F_1.tif"
raster= r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\02-satellite\Makkah-WV2-ST-2023_I602488\Makkah-WV2-ST-2023-MOS\23JAN19075627-S2AM-Makkah-WV2-ST-2023-tiled.TIF"
# aoi = "/Users/philipp/BPLA Dropbox/03 Planning/1232-T2-GIS and Remote Sensing-TMO/02_GIS_Data/LandCoverClassificaiton/lcc-trial/aoi-lcc.geojson"
training = r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\mekka-training-tree-0.shp"

In [4]:
# read training polygones
train = gpd.read_file(training) # this may contain multipolygons
train = train.to_crs(32637)
train_ex = train.explode()
train_ex.geometry=train_ex.centroid # we should use the full range of values inside the polygon not jsust the centroid 
# train_ex['lon'] = train_ex.centroid.x  
# train_ex['lat'] = train_ex.centroid.y
# TODO: https://www.matecdev.com/posts/random-points-in-polygon.html
# aoi = geopandas.read_file(aoi)

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

Can't load r

In [5]:
with rasterio.open(raster, 'r') as ds:
    arr = ds.read()  # read all raster values
    transform = ds.transform 
    crs = ds.crs
    res=ds.res

print(arr.shape)
print(crs)
print(res)
print(transform)

In [ ]:
# Transform the CRS to WGS84 lat/lon
train_ex = train_ex.to_crs('epsg:32637')

# with gw.open(raster) as src:
#     df = src.gw.extract(train_ex)
with gw.config.update(sensor='bgr'):
    with gw.open(raster) as src:
        df = src.gw.extract(
            train_ex,
            band_names=src.band.values.tolist()
        )